# JetRacerデータ収集と学習
このノートブックでは、JetRacerの自動走行に必要なデータを収集して、学習と予測値の確認をおこないます。   

## JetRacerデータ収集
JetRacerのデータ収集は以下の手順になります。  
  * カメラを起動する
  * カメラ映像をクリック可能なwidgetに表示する
  * カメラwidgetをクリックする
  * クリックしたx,y座標をファイル名に含めて画像をjpeg形式で保存する
  * 収集したデータの座標を編集可能にする

まずはこのノートブックの実行に必要なパッケージを読み込みます。

In [ ]:
# IPython Libraries for display and widgets
import ipywidgets
import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets
from traitlets import observe
from jupyter_clickable_image_widget import ClickableImageWidget

# Camera and Motor Interface
from jetcam.utils import bgr8_to_jpeg
from jetracer.utils import preprocess

# Python basic pakcages for image annotation
import cv2
import glob
import numpy as np
import os
import PIL.Image
import re
import threading
import time
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

# UNIXTIMEのミリセカンド表記を取得するためのラムダ関数を定義します
current_milli_time = lambda: int(round(time.time() * 1000))

カメラトラブル回避のために、一度カメラ用のデーモンを再起動しておきます。

In [ ]:
!echo jetson | sudo -S systemctl restart nvargus-daemon

カメラクラスをインスタンス化します。  
widgetでのカメラ画像の表示は遅いため、widgetの負担を少なくするためにfpsは小さくしておきます。

In [ ]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=224, height=224, capture_fps=10)

camera.running = True

データセット用のディレクトリを作成します。

In [ ]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

Jupyterノートブックでは、前のセルで定義した関数内のprint()は表示することができないため、デバッグが困難になります。  
そのため、ログ出力用のウィジェットを作成してprint()の代わりにログを表示します。

In [ ]:
# 汎用レイアウトを定義
description_style = {'description_width': 'initial'}
widget_width = ipywidgets.Layout(width=str(camera.width)+'px')
widget_width_half = ipywidgets.Layout(width=str(camera.width/2)+'px')

# ログ表示用ウィジェット
process_layout = ipywidgets.Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=process_layout, style=description_style)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

display(process_widget)

write_log("ログを表示")

カメラウィジェットをクリックしてデータを保存したときに、学習用のデータセットの情報も更新しておきたいので、カメラウィジェットを作る前にデータセットを定義します。

In [ ]:
def get_xy(name):
    """
    name:
        dir/xy_unixtimemillisec_x_y.jpg
        xy_unixtimemillisec_x_y.jpg
    """
    pattern = '.*xy_(\d+)_(\d+)_(\d+).*'
    result = re.match(pattern, name)
    if result:
        millisec = result.group(1)
        x = result.group(2)
        y = result.group(3)
    else:
        millisec = 0
        x = 0
        y = 0
    return x, y

def get_millisec(name):
    """
    name:
        xy_unixtimemillisec_x_y.jpg
    """
    pattern = '^xy_(\d+)_(\d+)_(\d+).*'
    result = re.match(pattern, name)
    if result:
        millisec = result.group(1)
        x = result.group(2)
        y = result.group(3)
    else:
        millisec = 0
        x = 0
        y = 0
    return millisec

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.color_jitter = transforms.ColorJitter(0.2, 0.2, 0.2, 0.2)
        self.random_hflips = random_hflips
        self.refresh()
        
    def __len__(self):
        return len(self.annotations)

   
    def __getitem__(self, idx):
        image_path = self.annotations[idx]['image_path']
        x_ratio, y_ratio = get_xy(image_path)
        x = float(int(x_ratio)-50.0)/50.0
        y = float(int(y_ratio)-50.0)/50.0

        image = PIL.Image.open(image_path)

        # ランダムに画像を水平反転する時、出力のxも対応するように反転します。デフォルトではFalseです。
        if self.random_hflips:
            if float(np.random.rand(1)) > 0.5:
                image = transforms.functional.hflip(image)
                x = -x

        # 画像をモデル学習の入力用データフォーマットに変換します
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        # ImageNetの正規化と同じパラメータでデータを正規化します
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

        return image, torch.tensor([x, y]).float()

    def refresh(self):
        self.annotations = []
        for image_path in glob.glob(os.path.join(self.directory, '*.jpg')):
            x, y = get_xy(image_path)
            self.annotations += [{
                'image_path': image_path,
                'x': x,
                'y': y
            }]

dataset = XYDataset(DATASET_DIR, random_hflips=False)

データ収集用のカメラウィジェットと確認用のスナップショットウィジェットを定義します。  

In [ ]:
# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ClickableImageWidget(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
no_widget = ipywidgets.IntText(description='no', style=description_style, layout=widget_width)

# create widgets
count_widget = ipywidgets.IntText(description='count', style=description_style, layout=widget_width)

# manually update counts at initialization
count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))


# カメラ画像を保存する機能を作成します
def save_snapshot(_, content, msg):
    global DATASET_DIR, dataset, snapshot_widget, no_widget
    if content['event'] == 'click':
        data = content['eventData']
        # クリックしたx,y座標を取得します（ピクセル座標）
        x = data['offsetX']
        y = data['offsetY']
        # パーセントに変換します
        x_ratio = int((x/224)*100)
        y_ratio = int((y/224)*100)
        # ファイル名を決定します
        filename = 'xy_%13d_%03d_%03d' % (current_milli_time(), x_ratio, y_ratio) + '.jpg'
        # 保存先をディレクトリパスを含めて決定します
        image_path = os.path.join(DATASET_DIR, filename)
        # 画像を保存します
        with open(image_path, 'wb') as f:
            f.write(camera_widget.value)

        # カメラ画像をsnapshot変数にコピーします
        snapshot = camera.value.copy()
        # クリックした座標に緑色で丸を描きます
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        # OpenCV BGR画像をjpeg画像に変換して、スナップショットウィジェットに表示します
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        # データファイル総数を確認します
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
        # ナンバーウィジェットにデータファイル総数を表示します
        no_widget.value = count_widget.value
        # データセットクラスが持つデータ情報を更新します
        dataset.refresh()

camera_widget.on_msg(save_snapshot)

スナップショットウィジェットで座標を編集できるようにします。

In [ ]:
def load_img(no):
    """
    noは1からn番までの値で、ファイル番号を表します。
    """
    global DATASET_DIR, img,load_flag,no_widget,snapshot_widget
    filenames = os.listdir(DATASET_DIR)
    filenames.sort()
    if len(filenames) == 0:
        no_widget.value = 0
        write_log("データファイルが存在しません。")
        return
    if no > len(filenames):
        no = 1
    if no < 1:
        no = len(filenames)

    no_widget.value = no
    name = filenames[no -1]
    write_log(str(no) + "枚目の" + name + "を読込みます。")
    
    x_ratio, y_ratio = get_xy(name)
    x = int(float(x_ratio)/100*224)
    y = int(float(y_ratio)/100*224)
    write_log("x,y,name: {},{}, {}".format(x,y,name))
    img = cv2.imread(os.path.join(DATASET_DIR, name))
    marked_img = img.copy()
    marked_img = cv2.circle(marked_img, (int(x), int(y)), 8, (0, 255, 0), 3)
    snapshot_widget.value = bgr8_to_jpeg(marked_img)
    write_log(str(no) + "枚目の" + name + "を読込みました。")

def prev_pic(c):
    global img,load_flag
    load_flag = True
    no = no_widget.value
    no = int(no) - 1
    load_img(no)

def next_pic(c):
    global x,y,load_flag
    load_flag = True
    no = no_widget.value
    no = int(no) + 1
    load_img(no)

def save_edit(_, content, msg):
    global DATASET_DIR,dataset,img,x,y,load_flag,snapshot_widget,no_widget
    if content['event'] == 'click' and load_flag == True:
        #load_flag = False
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        # save to disk
        #dataset.save_entry(category_widget.value, camera.value, x, y)
        x_ratio = int((x/224)*100)
        y_ratio = int((y/224)*100)

        filenames = os.listdir(DATASET_DIR)
        filenames.sort()
        old_file_name = filenames[no_widget.value -1]
        old_file_path = os.path.join(DATASET_DIR, old_file_name)
        write_log("old_file_path: {}".format(old_file_path))
        millisec = int(get_millisec(old_file_name))
        new_file_name = 'xy_%13d_%03d_%03d' % (millisec, x_ratio, y_ratio) + '.jpg'
        new_file_path = os.path.join(DATASET_DIR, new_file_name)
        write_log("new_file_path: {}".format(new_file_path))
        os.rename(old_file_path, new_file_path)
        
        # display saved remarked_img
        remarked_img = cv2.imread(new_file_path)
        remarked_img = cv2.circle(remarked_img, (int(x), int(y)), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(remarked_img)
        #count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
        dataset.refresh()
        write_log("新しい座標で保存しました。")


snapshot_widget.on_msg(save_edit)
prev_pic_button = ipywidgets.Button(description='prev', layout=widget_width_half)
next_pic_button = ipywidgets.Button(description='next', layout=widget_width_half)

prev_pic_button.on_click(prev_pic)
next_pic_button.on_click(next_pic)

左側にデータ収集用のカメラウィジェット、右側に編集機能付きのスナップショットウィジェットを表示します。  
左側のカメラ映像をクリックすると、その時のx,y座標と画像をjpegファイルで保存します。  
また、右側のウィジェットに保存した画像とクリックした座標を緑丸で表示します。  

保存する画像ファイル名は、編集時のファイル読み込み順を固定するためのソート用としてUNIXTIMEのミリ秒表記も含めておきます。  
また、画像サイズを変更しても同様の動作となるようにx,y座標はピクセル座標ではなく、パーセント表記に変換した値を使うことにします。
> JetRacerは学習する左右の値はステアリング値として使用します。上下の値は学習はしていますが、自動走行には使用していません。自動走行用の多少改修することで上下の値をスロットル値として利用可能にすることもできます。  

In [ ]:
# スナップショットウィジェットと説明と表示中のファイル番号とファイル操作ボタンを垂直に配置します
vb_snapshot_widget = ipywidgets.VBox([
    snapshot_widget,
    ipywidgets.Label('edit data'),
    no_widget,
    ipywidgets.HBox([prev_pic_button, next_pic_button])],
    layout=ipywidgets.Layout(align_items='center')
)

display(
    # ウィジェットを水平に配置します
    ipywidgets.HBox([
        # カメラウィジェットと説明とファイル総数を垂直に配置します
        ipywidgets.VBox([
            camera_widget,
            ipywidgets.Label('click to collect data'),
            count_widget
        ], layout=ipywidgets.Layout(align_items='center')),
        vb_snapshot_widget
    ]),
    # 最後にログ表示ウィジェットを配置します
    process_widget)

## 学習モデルを定義します
モデルはresnet18をベースにして、予測する値はx,yの2つとなるので、`model.fc`を変更します。  

In [ ]:
device = torch.device('cuda')
output_dim = 2  # x, y coordinate

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# DENSENET 121
# model = torchvision.models.densenet121(pretrained=True)
# model.classifier = torch.nn.Linear(model.num_features, output_dim)

model = model.to(device)

モデル読込み、保存用のボタンを作成します。

In [ ]:
model_save_button = ipywidgets.Button(description='save model', layout=widget_width_half)
model_load_button = ipywidgets.Button(description='load model', layout=widget_width_half)
model_path_widget = ipywidgets.Text(description='model', value='road_following_model.pth', style=description_style, layout=widget_width)

def load_model(c):
    start = time.time()
    write_log(model_path_widget.value + "の読込処理を開始します。")
    model.load_state_dict(torch.load(model_path_widget.value))
    process_time = time.time() - start
    write_log(model_path_widget.value + "の読込処理を終了しました(処理時間:{0:.3f}秒)。".format(process_time))
model_load_button.on_click(load_model)
    
def save_model(c):
    start = time.time()
    write_log(model_path_widget.value + "の書込処理を開始します。")
    torch.save(model.state_dict(), model_path_widget.value)
    process_time = time.time() - start
    write_log(model_path_widget.value + "の書込処理を終了しました(処理時間:{0:.3f}秒)。".format(process_time))
model_save_button.on_click(save_model)

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button]),
])

## 予測結果を表示するウィジェットを作成
学習後にシームレスに成果を確認するために、結果を表示するためのウィジェットを作成しておきます。

In [ ]:
state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop', style=description_style)
state_widget.style.button_width='50px'
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

def live(state_widget, model, camera, prediction_widget):
    while state_widget.value == 'live':
        image = camera.value
        preprocessed = preprocess(image)
        output = model(preprocessed).detach().cpu().numpy().flatten()
        x_pred = output[0]
        y_pred = output[1]
        #write_log("x_pred,y_pred = {},{}".format(x_pred,y_pred))
        x_ratio = x_pred/2 + 0.5
        y_ratio = y_pred/2 + 0.5
        #write_log("x_ratio,y_ratio = {},{}".format(x_ratio,y_ratio))
        x = int(camera.width * x_ratio)
        y = int(camera.height * y_ratio)
        #write_log("x,y = {},{}".format(x,y))

        prediction = image.copy()
        prediction = cv2.circle(prediction, (int(x), int(y)), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
            
def start_live(change):
    if change['new'] == 'live':
        write_log("liveモードを開始します。")
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget))
        execute_thread.start()
    else:
        write_log("liveモードを停止します。")

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    ipywidgets.Label('live prediction'),
    state_widget
], layout=ipywidgets.Layout(align_items='center'))

## 学習

In [ ]:
# 一回のミニバッチ処理で読込むデータ件数を定義します
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

# 学習用のウィジェットを定義します
epochs_widget = ipywidgets.IntText(description='epochs', value=1, style=description_style, layout=widget_width)
eval_button = ipywidgets.Button(description='evaluate', layout=widget_width_half)
train_button = ipywidgets.Button(description='train', layout=widget_width_half)
loss_widget = ipywidgets.FloatText(description='loss', style=description_style, layout=widget_width)
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress', style=description_style, layout=widget_width)


# プログレスバーを更新するクラスを定義します
class Progress(traitlets.HasTraits):
    value = traitlets.Float()
    total_value = 100

    @observe('value')
    def _on_value(self, change):
       
        self._update_progress()
    
    def _update_progress(self):
        global progress_widget
        progress_widget.value = self.value / self.total_value

# 学習と評価を実行する関数を定義します
def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, model_save_button, model_load_button, state_widget, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        dataset.refresh()
        progress_widget.value = 0

        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True,
            num_workers=0
        )

        # ライブ予測を停止します
        state_widget.value = 'stop'
        # 各種ボタン操作を無効化します
        model_save_button.disabled = True
        model_load_button.disabled = True
        state_widget.disabled = True
        train_button.disabled = True
        eval_button.disabled = True
        # ライブ予測スレッドが停止するまで1秒待ちます
        time.sleep(1)
        start = time.time()
        if is_training:
            model = model.train()
            write_log("{}Epochの学習を開始します。".format(epochs_widget.value))
        else:
            # ドロップアウトを無効にする評価モード
            model = model.eval()
            write_log("評価モードで学習を開始します(1 Epochのみ)。")

        epoch_num = 1
        # プログレスバーの更新を定義します
        progress = Progress()
        progress.total_value = 8*len(dataset)/BATCH_SIZE
        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            epoch_start = time.time()
            progress.value = 0
            for images, xy in iter(train_loader):
                progress.value += 1
                # send data to device
                images = images.to(device)
                progress.value += 1
                xy = xy.to(device)
                progress.value += 1
                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()
                progress.value += 1

                # execute model to get outputs
                outputs = model(images)
                progress.value += 1

                # compute MSE loss over x, y coordinates for associated categories
                loss = 0.0
                loss += F.mse_loss(outputs, xy)
                progress.value += 1

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()
                progress.value += 1

                # increment progress
                i += 1
                sum_loss += float(loss)
                loss_widget.value = sum_loss / i
                progress.value += 1
                
            if is_training:
                process_time = time.time() - epoch_start
                write_log(str(epoch_num)+"Epoch目の学習が終了しました(処理時間:{0:.3f}秒)。".format(process_time))
                epochs_widget.value -= 1
                epoch_num += 1
            else:
                break
    except e:
        pass
    model = model.eval()
    
    model_save_button.disabled = False
    model_load_button.disabled = False
    state_widget.disabled = False
    train_button.disabled = False
    eval_button.disabled = False
   
    process_time = time.time() - start
    if is_training:
        write_log("すべての学習が終了しました(トータルの処理時間:{0:.3f}秒)。".format(process_time))
    else:
        write_log("すべての評価が終了しました(トータルの処理時間:{0:.3f}秒)。".format(process_time))
        
    state_widget.value = 'live'
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

## それでは学習しましょう！
最初は1エポックで学習します。ここでプログラムが動作するか確認することができます。  
次に、10エポックで学習します。ライブ予測画面に青丸が表示されることを確認してください。この青丸が予測したx,y座標になります。  

そこまで確認できたら、あとはデータを取って学習して、うまく精度が出ないところを重点的にデータを集めることで、精度をあげることができます。

In [ ]:

vb_data_collection_widget = ipywidgets.VBox([
        camera_widget,
        ipywidgets.Label('click to collect data'),
        count_widget,
        train_eval_widget,
        model_widget,
    ], layout=ipywidgets.Layout(align_items='center'))



all_widget = ipywidgets.VBox([

    ipywidgets.HBox([vb_data_collection_widget,
                     vb_snapshot_widget,
                     live_execution_widget]), 

    process_widget
])

display(all_widget)

## 次の作業

convert_to_trt.ipynb で学習済みモデルとTensorRT形式に変換します。